In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

import plotly.graph_objects as go
import plotly.express as px

import dash
from dash import dcc
from dash import html

df = pd.read_csv("sample.csv")
df.columns =["EPF","Name","SatisfactionLevel","ReasonCategory","LTO"]
#df.drop([0] , inplace =True)
# Creating dummy variables
df_dummies = pd.get_dummies(df, columns=["ReasonCategory"])
print(df_dummies)


# Standardize features

# Split the data into training and testing sets
X = df_dummies[["ReasonCategory_A","ReasonCategory_B","ReasonCategory_C","ReasonCategory_D","ReasonCategory_E","ReasonCategory_F","SatisfactionLevel"]]
y = df_dummies['LTO']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Fit the logistic regression model
model = LogisticRegression()
model.fit(X_train_scaled, y_train)
coefficients = model.coef_[0]  # For binary classification, coef_ is a 2D array with shape (1, n_features)
intercept = model.intercept_[0]
print(coefficients)
print(intercept)

# Predict probabilities
y_pred = model.predict_proba(X_test_scaled)[:, 1]

# Sample data
np.random.seed(0)
df = df_dummies
# Logistic regression
X = df[['SatisfactionLevel']]
y = df['LTO']

# Fit the model
model = LogisticRegression()
model.fit(X, y)

# Generate predictions
x_range = np.linspace(df['SatisfactionLevel'].min(), df['SatisfactionLevel'].max(), 5)
y_proba = model.predict_proba(x_range.reshape(-1, 1))[:, 1]

# Create the plot
fig = go.Figure()

# Scatter plot of actual data points
fig.add_trace(go.Scatter(x=df['SatisfactionLevel'], y=df['LTO'], mode='markers', name='Data'))

# Logistic regression curve
fig.add_trace(go.Scatter(x=x_range, y=y_proba, mode='lines', name='Logistic Regression'))

# Customize layout
fig.update_layout(
    title='LTO susceptibility curve',
    xaxis_title='Satisfaction Level',
    yaxis_title='Probability of Getting LTO',
    showlegend=True
)

fig.show()

# Sample data
df = df
# Scatter plot
scatter_fig = px.scatter(df, x='EPF', y='LTO', color='SatisfactionLevel',
                         title="Scatter plot of Satisfaction Level vs LTO Suceptibility")

 #Histogram
hist_fig = px.histogram(df, x='SatisfactionLevel', title="Histogram of Employee Satisfaction Level")


# Initialize Dash app
app = dash.Dash(__name__)

# Define the layout of the dashboard
app.layout = html.Div(children=[
    html.H1(children='LTO Susceptibility'),

    html.Div(children='OVERVIEW OF LTO TREND'),

    dcc.Graph(
        id='scatter-plot',
        figure=scatter_fig
    ),

    dcc.Graph(
        id='histogram',
        figure=hist_fig
    )
])

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

FileNotFoundError: [Errno 2] File sample.csv does not exist: 'sample.csv'